# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [76]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [77]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [78]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [79]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Working with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Writing Apache Cassandra code in the cells below

#### Creating a Cluster

In [80]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [81]:
#Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [82]:
#Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Creating tables to run the following queries.

## Create queries to ask the following three questions of the data

### 1. artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3.every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## TABLE 1: artists_songs_length_by_session

### For table artists_songs_length_by_session, sessionId and itemInsession were used as a partition key because the queries will filter by this column. 


In [83]:
## Query 1:  artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query = "CREATE TABLE IF NOT EXISTS artists_songs_length_by_session "
query = query + """(sessionId int,
                    itemInSession int,
                    artist text,
                    song text,
                    length float,                    
                    PRIMARY KEY (sessionId, itemInsession))"""
try:
    session.execute(query)
except Exception as e:
    print(e)                 

In [84]:
# complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    count = 0
    for line in csvreader:
##Assign the INSERT statements into the `query` variable
        query = "INSERT INTO artists_songs_length_by_session (sessionId,itemInSession,artist, song, length)"
        query = query +  " VALUES (%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        try:
            session.execute(query, (int(line[-3]),int(line[3]),line[0],line[-2],float(line[5])))
        except Exception as e:
            print(e)
        #print(line)
        #print(line[0],line[-2],line[5],line[3],line[-3])
        #count = count +1
        #if count==5:
        #    break

#### SELECT to verify that the data have been inserted into each table

In [85]:
##Add in the SELECT statement to verify the data was entered into the table
query = "select artist,song,length from artists_songs_length_by_session WHERE sessionId = 338 and itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


## TABLE 2: artists_songs_users_by_userId_session

### For table artists_songs_users_by_userId_sessiony, userId and sessionId were used as a partition key because the queries will filter by this column. itemInSession was used as a clustering column to help make up a unique key.

In [86]:
## Query 2: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = "CREATE TABLE IF NOT EXISTS artists_songs_users_by_userId_session"
query = query + """(userId int,
                    sessionId int,
                    itemInSession int,
                    artist text,
                    song text,
                    user text,    
                    PRIMARY KEY ((userId, sessionId),itemInSession))"""
try:
    session.execute(query)
except Exception as e:
    print(e) 

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    count = 0
    for line in csvreader:
        query = "INSERT INTO artists_songs_users_by_userId_session (userId, sessionId,itemInSession, artist, song, user)"
        query = query +  " VALUES (%s, %s, %s, %s, %s, %s)"
        try:
            session.execute(query, (int(line[-1]),int(line[-3]),int(line[3]),line[0],line[-2],line[1]+line[4]))
        except Exception as e:
            print(e)
        #name = line[1]+line[4]
        #print(line)
        #print(line[0],line[-2],line[1]+line[4],int(line[3]),int(line[-1]),int(line[-3]))
        #count = count +1
        #if count==5:
        #    break 
query = "select artist,song,user from artists_songs_users_by_userId_session WHERE userId = 10 and sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.user)


Down To The Bone Keep On Keepin' On SylvieCruz
Three Drives Greece 2000 SylvieCruz
Sebastien Tellier Kilometer SylvieCruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) SylvieCruz


## TABLE 3: users_per_song

### For table users_per_song, song and userId userId were used as a partition key because the queries will filter by this column.

In [87]:
##every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = "CREATE TABLE IF NOT EXISTS users_per_song"
query = query + """(song text,
                    userId int,
                    user text,
                    PRIMARY KEY (song,userId))"""
try:
    session.execute(query)
except Exception as e:
    print(e) 

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    count = 0
    for line in csvreader:
        query = "INSERT INTO users_per_song (song, userId,user)"
        query = query +  " VALUES (%s, %s,%s)"
        try:
            session.execute(query, (line[-2],int(line[-1]),line[1]+line[4]))
        except Exception as e:
            print(e)
        #name = line[1]+line[4]
        #print(line)
        #print(line[0],line[-2],line[1]+line[4],int(line[3]),int(line[-1]),int(line[-3]))
        #count = count +1
        #if count==5:
        #    break 
query = "select user from users_per_song WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.user)

                    

JacquelineLynch
TeganLevine
SaraJohnson


### Drop the tables before closing out the sessions

In [73]:
##Drop the table before closing out the sessions

In [74]:
query = "drop table IF EXISTS artists_songs_length_by_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table IF EXISTS artists_songs_users_by_userId_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "drop table IF EXISTS users_per_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [75]:
session.shutdown()
cluster.shutdown()